# Baseline model from police strategy

#### Setting environment 

In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
sys.path.append(os.path.join('c:', os.sep, 'whatever', 'path', 'to', 'sumo', 'tools'))

In [2]:
import traci
from sumolib import checkBinary
import traci.constants as tc
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
# import torch
import copy
import csv
import pandas as pd

In [3]:
# reward_memory = []
# def plot_durations():
#     print('show')
#     plt.figure(2)
#     plt.clf()
#     durations_t = torch.tensor(reward_memory, dtype=torch.float)
#     plt.title('Training...')
#     plt.xlabel('Step')
#     plt.ylabel('Reward')
#     plt.plot(durations_t.numpy())
#     # Take 100 episode averages and plot them too
#     if len(durations_t) >= 100:
#         means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
#         means = torch.cat((torch.zeros(99), means))
#         plt.plot(means.numpy())

#     plt.pause(0.001)  # pause a bit so that plots are updated

Threshold setting

In [4]:
# decision_threshold

threshold = {'ONNUT_FP1_END_FLOW' : 10,
'ONNUT_FP1_END_UNJAM' : 105, #30% of sum lenghth of edge infront e2detector ID=15
'ONNUT_FP2_END_FLOW' :  0.3, #20

#======= fixed ==========#
'VIRTUAL_FP1_END_UNJAM_17' : 80, #60
'VIRTUAL_FP1_END_UNJAM_13_14' : 30,
'VIRTUAL_FP1_END_UNJAM_BANGNA' : 20
            }


duration={ 
'ONNUT_FP1_END_MAXGREEN' : 90,
'ONNUT_FP2_END_MAXGREEN' : 90,
'ONNUT_FP1_END_MINGREEN': 15,
'ONNUT_FP2_END_MINGREEN':15,
'VIRTUAL_FP1_END_MAXGREEN': 90,
'VIRTUAL_FP1_END_MINGREEN' : 15
}

In [5]:
# collect phase changing reason

count = {'ONNUT_FP1_END_FLOW' : 0,
'ONNUT_FP2_END_FLOW' :  0,
'ONNUT_FP2_END_FLOW' : 0,
'ONNUT_FP1_END_UNJAM' : 0,
'ONNUT_FP1_END_MAXGREEN' : 0,
'ONNUT_FP2_END_MAXGREEN' : 0,
'ONNUT_FP1_END_MINGREEN': 0,
'ONNUT_FP2_END_MINGREEN':0,
'VIRTUAL_ONNUT_ASSOCIATE':0,
'VIRTUAL_FP1_END_MINGREEN':0,
'VIRTUAL_FP1_END_MAXGREEN':0,
'VIRTUAL_FP1_END_UNJAM_17':0,
'VIRTUAL_FP1_END_UNJAM_13_14' : 0,
'VIRTUAL_FP1_END_UNJAM_BANGNA' : 0
        }

#### import data by csv files

In [6]:
# os.chdir('C:\\Users\\Asus\\OneDrive - Chulalongkorn University\\Desktop\\Senior Project\\Experiments-with-DQN-Onnut-27042022\\Experiments-with-DQN-Onnut-21042022\\multi-agent-dqn-main')
# namedetector_df = pd.read_csv('name_detector.csv')
# namedetector_df.dropna(inplace=True)

# # #namedetector_df

#### Start SUMO

In [7]:
def start(random_seed):
    # os.chdir('C:\\Users\\Asus\\OneDrive - Chulalongkorn University\\Desktop\\Senior Project\\Experiments-with-DQN-Onnut-27042022\\Experiments-with-DQN-Onnut-21042022\\multi-agent-dqn-main')
    sumoBinary = checkBinary('sumo-gui')
    # sumoBinary = checkBinary('sumo')
    # traci.start([sumoBinary, "-c", "onnut_ake.sumocfg",'--start','true','--quit-on-end','true','--time-to-teleport','-1'])
    traci.start([sumoBinary, "-c", "onnut_ake.sumocfg",'--start','true','--quit-on-end','true','--time-to-teleport','900',
            "--no-internal-links",'true',"--ignore-junction-blocker",'-1','--seed', str(random_seed)])

#### Define environment

In [8]:
def get_occupancy_average_percent(detector_id): 
    #get occupancy average for all detector in detector_id and scale by (Vehicle Length + MinimumGap)/MinimumGap 
    #Vehicle Length = 4.62 MinimumGap = 2.37
    occupancy = (sum([traci.lanearea.getLastStepOccupancy(e) for e in detector_id])/len(detector_id))*((4.62+2.37)/4.62)
    return occupancy

In [9]:
def get_flow_sum(detector_id):
#     Speed (metres per sec) = flow (vehicle per sec) / density (veh per metre), Ajarn chaodit
#         flow= int(densityPerLane) * float(meanSpeed)#flow per lane
#     print('LastStepVehicleNumber', sum([traci.lanearea.getLastStepVehicleNumber(e) for e in detector_id]))
#     print('length', sum([traci.lanearea.getLength(i) for i in detector_id]))
#     density = sum([traci.lanearea.getLastStepVehicleNumber(e) for e in detector_id])/\
#     sum([traci.lanearea.getLength(i) for i in detector_id])
#     print('density', density)
    
    flow = sum(([traci.lanearea.getLastStepVehicleNumber(e)*traci.lanearea.getLastStepMeanSpeed(e)/traci.lanearea.getLength(e) for e in detector_id if
              traci.lanearea.getLastStepMeanSpeed(e) >= 0]))
            
    return flow

In [10]:
def get_unjamlength_meters(detector_id): 
    detector_length = sum(traci.lanearea.getLength(d) for d in detector_id)
    unjamlength = detector_length - (sum([traci.lanearea.getJamLengthMeters(e) for e in detector_id])) #/detector_length
    return unjamlength

#### Define Reward

In [11]:
def get_throughput_onnut():
    #loopID = ['L751454884#3_0','L751454884#3_1','L751454884#3_2','L156591171#2_0','L156591171#2_1','L156591171#2_2','L824116560#3_0','L824116560#3_1','L824116560#3_2','L824116560#3_3','L-93727606#1_0']
    loopID = ['Induction_Loop_1','Induction_Loop_2','Induction_Loop_3','Induction_Loop_4','Induction_Loop_5','Induction_Loop_6','Induction_Loop_7','Induction_Loop_8','Induction_Loop_9','Induction_Loop_10']
    # throughput_onnut = sum([traci.inductionloop.getLastStepVehicleNumber(i) for i in loopID if traci.inductionloop.getLastStepMeanSpeed(i) > 0])  #if traci.inductionloop.getLastStepMeanSpeed(i) > 0
    throughput_onnut = 0
    for id in loopID:

        laneID = traci.inductionloop.getLaneID(id)
        edgeID = traci.lane.getEdgeID(laneID)
        speed = traci.edge.getLastStepMeanSpeed(edgeID)
        if speed > 0:
            throughput_onnut += traci.inductionloop.getLastStepVehicleNumber(id)

    return throughput_onnut

In [12]:
def get_throughput_virtual():
    loopID = ["Virtual_loop_1","Virtual_loop_2","Virtual_loop_3","Virtual_loop_4","Virtual_loop_5"]
    throughput_virtual = 0
    for id in loopID:

        laneID = traci.inductionloop.getLaneID(id)
        edgeID = traci.lane.getEdgeID(laneID)
        speed = traci.edge.getLastStepMeanSpeed(edgeID)
        if speed > 0:
            throughput_virtual += traci.inductionloop.getLastStepVehicleNumber(id)

    return throughput_virtual

In [13]:
def get_travel_time_onnut() :
    edgeID_for_MOE = [  #NB
                '824116560#0','824116560#1','824116560#2','824116560#3','824816455','220429932#0','824116561-AddedOffRampEdge','113135465#5',
                '750035412#1-AddedOffRampEdge','824816456','113135465#0','113135465#2',
                #SB
                '751454884#3','751454884#2','751454884#0',
                '751454885#0','751454885#2','751454885#3','751454885#5',
                #WB
                '824456410#0','824456410#2','824456410#2.41',
                '156591171#2','-824456410#2','156591171#0'
                                ]

    if ((sum(traci.edge.getLastStepVehicleNumber(edgeID) for edgeID in edgeID_for_MOE)) != 0) : 
        travel_time_onnut = sum(traci.edge.getTraveltime(edgeID)*traci.edge.getLastStepVehicleNumber(edgeID) for edgeID in edgeID_for_MOE) \
            /(sum(traci.edge.getLastStepVehicleNumber(edgeID) for edgeID in edgeID_for_MOE) *len(edgeID_for_MOE))
    else : 
        travel_time_onnut = 0

    return travel_time_onnut 

In [14]:
def get_travel_time_virtual() :
    edgeID_for_MOE = [  #SB
                '824456410#2','824456410#2.41','824456409#0','-gneE25',
                '-824456410#2','156591171#0','-824456409#0','-gneE24',
                #WB
                'gneE34','824456409#5','824456409#6','113135397#1','113135397#3',
                'gneE33','-824456409#5','-113135397#0','-113135397#2','-113135397#4',
                #BigC
                'gneE32','gneE29'
                                ]
                                
    if ((sum(traci.edge.getLastStepVehicleNumber(edgeID) for edgeID in edgeID_for_MOE)) != 0) : 
        travel_time_virtual = sum(traci.edge.getTraveltime(edgeID)*traci.edge.getLastStepVehicleNumber(edgeID) for edgeID in edgeID_for_MOE) \
            /(sum(traci.edge.getLastStepVehicleNumber(edgeID) for edgeID in edgeID_for_MOE) *len(edgeID_for_MOE))
    else : 
        travel_time_virtual = 0

    return travel_time_virtual

In [15]:
def get_waiting_time_onnut() :
    edgeID = [  #NB
                '824116560#0','824116560#1','824116560#2','824116560#3','824816455','220429932#0','824116561-AddedOffRampEdge','113135465#5',
                '750035412#1-AddedOffRampEdge','824816456','113135465#0','113135465#2',
                #SB
                '751454884#3','751454884#2','751454884#0',
                '751454885#0','751454885#2','751454885#3','751454885#5',
                #WB
                '824456410#0','824456410#2','824456410#2.41',
                '156591171#2','-824456410#2','156591171#0'
                                ]
    waiting_time_onnut = sum([traci.edge.getWaitingTime(i) for i in edgeID])/len(edgeID)
    return waiting_time_onnut 

In [16]:
def get_waiting_time_virtual() :
    edgeID = [  #SB
                '824456410#2','824456410#2.41','824456409#0','-gneE25',
                '-824456410#2','156591171#0','-824456409#0','-gneE24',
                #WB
                'gneE34','824456409#5','824456409#6','113135397#1','113135397#3',
                'gneE33','-824456409#5','-113135397#0','-113135397#2','-113135397#4',
                #BigC
                'gneE32','gneE29'
                                ]
    waiting_time_virtual = sum([traci.edge.getWaitingTime(i) for i in edgeID])/len(edgeID)
    return waiting_time_virtual

#### Define Action

Onnut intersection ID & Virtual intersection ID

In [17]:
# Onnut junction ID : cluster_1088409501_272206263_5136790697_70702637
# Virtual junction ID : : gneJ42 (old : gneJ42_4_1)

id = {'onnut_junction':'cluster_1088409501_272206263_5136790697_70702637',
      'virtual_junction':'gneJ42'}
junction_name = list(id.keys())

In [18]:
#traci.trafficlight.getCompleteRedYellowGreenDefinition('cluster_1088409501_272206263_5136790697_70702637')

Select & take action

In [19]:
def find_action(selected_traffic_signal_phases,list_greentime_duration, threshold, duration): 
#     ==
# Function traffic_police_action takes inputs, namely, 

# selected_traffic_signal_phases[even index] = current phase of traffic light signal from previous decision time step
# selected_traffic_signal_phases[odd index] = next phase of traffic light signal from previous decision time step if no signal interruption would occur
# greentime_duration = number of seconds that current phase of traffic light signal has been used
# max_greentime_duration = maximum allowable number of seconds for current phase of traffic light signal

# and returns outputs, namely,

# new_selected_traffic_signal_phases = new value of selected_traffic_signal_phases[0] and selected_traffic_signal_phases[1] (defined above) to be deployed till next decision time step
# new_greentime_duration = new value of greentime_duration to be deployed till next decision time step
#     ==


    ONNUT_greentime_duration,ONNUT_phase, next_ONNUT_phase_ifno_interupt = ONNUT_find_action(selected_traffic_signal_phases,list_greentime_duration,\
                                                                                             threshold, duration)
    
    VIRTUAL_greentime_duration,VIRTUAL_phase, next_VIRTUAL_phase_ifno_interupt = VIRTUAL_find_action(selected_traffic_signal_phases,list_greentime_duration,\
                                                                                             threshold, duration,ONNUT_phase)
#     PHRAKHANONG_SUKHUMVIT71_greentime_duration,PHRAKHANONG_SUKHUMVIT71_phase, next_PHRAKHANONG_SUKHUMVIT71_phase_ifno_interupt = PHRAKHANONG_SUKHUMVIT71_find_action(selected_traffic_signal_phases,list_greentime_duration, \
#                                                                                                                threshold, duration)
    
#     PHRAKHANONG_phase = PHRAKHANONG_SUKHUMVIT71_phase
#     SUKHUMVIT71_phase = PHRAKHANONG_SUKHUMVIT71_phase
    return [ONNUT_phase,next_ONNUT_phase_ifno_interupt, VIRTUAL_phase,next_VIRTUAL_phase_ifno_interupt],[ONNUT_greentime_duration,VIRTUAL_greentime_duration] #, PHRAKHANONG_phase, None, SUKHUMVIT71_phase, None], [ONNUT_greentime_duration, PHRAKHANONG_SUKHUMVIT71_greentime_duration]

In [20]:
def ONNUT_find_action(selected_traffic_signal_phases,list_greentime_duration, threshold, duration):
#Onnut
    greentime_duration = list_greentime_duration[0]
    current_phase = selected_traffic_signal_phases[0]
    # print('onnut_phase_{}'.format(current_phase))
    next_phase_ifno_interupt = selected_traffic_signal_phases[1]
    infront_sukhumvit50_detection_id  = ['S_ONT_04_0','S_ONT_04_1','S_ONT_04_2'] #ID = 04
    infront_bigC_detection_id = ['S_ONT_15_0','S_ONT_15_1'] #ID = 15

    onnut_SB_detection_id = ['S_ONT_05_0','S_ONT_05_1']

    # print('onnut suk50 flow = {}'.format(get_flow_sum(infront_sukhumvit50_detection_id)))
    # print('===================================')
    if current_phase == 1 :

        if duration['ONNUT_FP1_END_MINGREEN'] >= greentime_duration :
            count['ONNUT_FP1_END_MINGREEN'] +=1 
            current_phase = 1

        elif duration['ONNUT_FP1_END_MAXGREEN'] <= greentime_duration :
            count['ONNUT_FP1_END_MAXGREEN'] +=1 
            current_phase = 2

        #elif 04 detector flow/occupancy??? 
        elif get_flow_sum(infront_sukhumvit50_detection_id) >= threshold['ONNUT_FP1_END_FLOW'] : 
            count['ONNUT_FP1_END_FLOW'] += 1
            current_phase = 2

        #elif infront Big c detector unjamed length???
        elif get_unjamlength_meters(infront_bigC_detection_id) <= threshold['ONNUT_FP1_END_UNJAM'] :
            count['ONNUT_FP1_END_UNJAM'] += 1
            current_phase = 2
 
    elif current_phase == 2 :
        if duration['ONNUT_FP2_END_MINGREEN'] >= greentime_duration :
            count['ONNUT_FP2_END_MINGREEN'] +=1 
            current_phase = 2
        elif duration['ONNUT_FP2_END_MAXGREEN'] <= greentime_duration :
            count['ONNUT_FP2_END_MAXGREEN'] +=1 
            current_phase = 1
            
        #elif 05detector flow/occupancy???
        elif get_flow_sum(onnut_SB_detection_id) <= threshold['ONNUT_FP2_END_FLOW'] :
            count['ONNUT_FP2_END_FLOW'] += 1
            current_phase = 1

    if selected_traffic_signal_phases[0] == current_phase:
        greentime_duration += 15
    else: greentime_duration = 0
    
    Onnut_phase = current_phase
    next_Onnut_phase_ifno_interupt = next_phase_ifno_interupt
    new_greentime_duration = greentime_duration
#     action = [ KlongTei_phase, None, Rama4_phase,  next_Rama4_phase, 0, None, 0, None, 0, None, 0, None]

    return  new_greentime_duration, Onnut_phase, next_Onnut_phase_ifno_interupt


In [21]:
def VIRTUAL_find_action(selected_traffic_signal_phases,list_greentime_duration, threshold, duration,ONNUT_phase) :
    #Virtual intersection infront of Big-C
    greentime_duration = list_greentime_duration[1]
    # print('virtual green time{}'.format(greentime_duration))
    current_phase = selected_traffic_signal_phases[2]
    # print('virtual_phase{}'.format(current_phase))
    next_phase_ifno_interupt = selected_traffic_signal_phases[3]

    onnut_next_phase = ONNUT_phase

    detector_id_17 = ['S_ONT_17_0']
    detector_id_15 = ['S_ONT_15_0','S_ONT_15_1']
    # detector_id_right_lane_get_in_BigC = ['S_ONT_13_1','S_ONT_14_1']
    detector_id_right_lane_get_in_BigC = ['S_ONT_13_1']
    detector_id_bangna_bridge = ['S_ONT_02_0','S_ONT_02_1']

    # print(get_unjamlength_meters(detector_id_17))
    # print(get_unjamlength_meters(detector_id_right_lane_get_in_BigC))
    # print(get_unjamlength_meters(detector_id_bangna_bridge))
    # print('==================================================')
    
    if onnut_next_phase == 2 :
        count['VIRTUAL_ONNUT_ASSOCIATE'] += 1
        #Associate phase condition for ONNUT & VIRTUAl intersection
        current_phase = 1

    elif current_phase == 1:
        # print(get_unjamlength_meters(detector_id_right_lane_get_in_BigC))

        if duration['VIRTUAL_FP1_END_MINGREEN'] >= greentime_duration :
            count['VIRTUAL_FP1_END_MINGREEN'] +=1 
            current_phase = 1

        #===============================================================#

        elif duration['VIRTUAL_FP1_END_MAXGREEN'] <= greentime_duration :
            count['VIRTUAL_FP1_END_MAXGREEN'] +=1 
            current_phase = 3

        #===============================================================#
        
        elif get_unjamlength_meters(detector_id_17) <= threshold['VIRTUAL_FP1_END_UNJAM_17'] :
            #Queue length in BigC
            count['VIRTUAL_FP1_END_UNJAM_17'] += 1
            current_phase = 2 

        elif get_unjamlength_meters(detector_id_right_lane_get_in_BigC) <= threshold['VIRTUAL_FP1_END_UNJAM_13_14']  :
            #Congested by car want to get in bigC (right lane)
            count['VIRTUAL_FP1_END_UNJAM_13_14'] += 1
            current_phase = 3 
        
        elif get_unjamlength_meters(detector_id_bangna_bridge) <= threshold['VIRTUAL_FP1_END_UNJAM_BANGNA'] : 
            #Onnut-Bangna Bridge condition 
            count['VIRTUAL_FP1_END_UNJAM_BANGNA'] += 1
            current_phase = 1

    elif current_phase == 2 :
        current_phase = 1 
    
    elif current_phase == 3 :
        current_phase = 1


    if selected_traffic_signal_phases[2] == current_phase:
        greentime_duration += 15
    else: greentime_duration = 0

    Virtual_phase = current_phase
    next_Virtual_phase_ifno_interupt = next_phase_ifno_interupt
    new_greentime_duration = greentime_duration

    return new_greentime_duration, Virtual_phase, next_Virtual_phase_ifno_interupt

In [22]:
def take_action(new_selected_traffic_signal_phases):
#     ==
# Function take_action takes inputs, namely, 

# new_selected_traffic_signal_phases[even indec] = current phase of traffic light signal from previous decision time step
# new_selected_traffic_signal_phases[odd index] = next phase of traffic light signal from previous decision time step if no signal interruption would occur

# and returns outputs, namely,

# JamLengthMeter = sum of jam length in detectors area
# Speed = sum of speed of all vehicles in detectors area


#======================= P'pupae ===========================#
    # change_to_phase = new_selected_traffic_signal_phases[0::2]

    # print(new_selected_traffic_signal_phases,change_to_phase)

    # change_to_phase_sumo = [e for e in change_to_phase]

    current_phase = [traci.trafficlight.getPhase(id[key]) for key in id.keys()]

#     for i in range(2):
# #         if change_to_phase[i] != current_phase[i]:
# #             yellow = (change_to_phase[i]-1)%(2*number_phase[i])
# # #             print(yellow)
# # #             print(junction_name[i])
# #             traci.trafficlight.setPhase(id[junction_name[i]], yellow) #  YELLOW PHASE 
# #         else:
#         # traci.trafficlight.setPhase(id[junction_name[i]], change_to_phase_sumo[i]) # No YELLOW PHASE 

#         traci.trafficlight.setPhase(id[junction_name[i]], new_selected_traffic_signal_phases[i])
    #print(new_selected_traffic_signal_phases)
    #Onnut intersection
    traci.trafficlight.setPhase(id[junction_name[0]], (new_selected_traffic_signal_phases[0]-1))

    #Virtual intersection 
    traci.trafficlight.setPhase(id[junction_name[1]], (new_selected_traffic_signal_phases[2]-1))

    throughput_onnut = 0
    throughput_virtual = 0

    
    waiting_time_onnut = 0
    waiting_time_virtual = 0

    teleport_number = 0 

    for i in range(15): # change to 10????
        throughput_onnut += get_throughput_onnut()
        throughput_virtual += get_throughput_virtual()

        teleport_number += traci.simulation.getEndingTeleportNumber()
        # waiting_time_onnut += get_waiting_time_onnut()
        # waiting_time_virtual += get_waiting_time_virtual()

        traci.simulationStep()

    travel_time_onnut = get_travel_time_onnut()
    travel_time_virtual = get_travel_time_virtual()
#     w1 = sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(reward_w1)])
#     w2 = sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(reward_w1)])

    # JamLengthPercent= sum([traci.lanearea.getJamLengthMeters(i) for i in list(namedetector_df['detector_id'])])/sum([traci.lanearea.getLength(i) for i in list(namedetector_df['detector_id'])])
    # Speed = sum([traci.lanearea.getLastStepMeanSpeed(i) for i in list(namedetector_df['detector_id'])])/(len(list(namedetector_df['detector_id'])))
    # Flow = sum([traci.lanearea.getLastStepVehicleNumber(e) for e in list(namedetector_df['detector_id'])])
    JamLengthPercent = 0
    Speed = 0
    Flow = 0
    
    
#     WaitingTime = sum([traci.lane.getWaitingTime(i) for i in lane])
#     HaltingNumber = sum([traci.lane.getLastStepHaltingNumber(i) for i in  lane]) #list(NAME['id'])
#     Traveltime = sum([traci.lane.getTraveltime(i) for i in  lane])
   
    return JamLengthPercent,Speed,Flow,throughput_onnut,throughput_virtual,travel_time_onnut,travel_time_virtual,waiting_time_onnut,waiting_time_virtual,teleport_number #,WaitingTime, HaltingNumber,  Traveltime 

#### Run baseline model

In [23]:
# with open( "./AkeOutput/onnut_occfull.csv" , 'w', newline='') as csv_file:
#     header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
#     writer = csv.DictWriter(csv_file, fieldnames = header)
#     writer.writeheader()
        
def episode0(threshold, duration,random_seed):
    #episode = 0
    action_onnut = []
    action_virtual = []
    reward_memory = []
    JamLengthPercent_memory = []
    Speed_memory = []
    throughput_onnut_memory = []
    throughput_virtual_memory = []
    travel_time_onnut_memory = []
    travel_time_virtual_memory = []
    waiting_time_onnut_memory = []
    waiting_time_virtual_memory = []
    teleport_number_memory = []

    list_greentime_duration = [0, 0]
    start(random_seed)
    traci.simulationStep()
    selected_traffic_signal_phases = [1, None,1 ,None]
    for seconds in range(960): #2880
#         state, len_state = get_state() #minimum_remaining_time = Q-lengthเมตร/free flow speed
        new_selected_traffic_signal_phases, new_greentime_duration = find_action(selected_traffic_signal_phases,
            list_greentime_duration, threshold, duration)
        # output = 2 lists with 2 components in each list.

#         print( new_selected_traffic_signal_phases, new_greentime_duration)
#         print(count)

        JamLengthPercent,Speed,Flow,throughput_onnut,throughput_virtual, \
        travel_time_onnut,travel_time_virtual,waiting_time_onnut,waiting_time_virtual,teleport_number \
        = take_action(new_selected_traffic_signal_phases)


        action_onnut.append(new_selected_traffic_signal_phases[0])
        action_virtual.append(new_selected_traffic_signal_phases[2])
        JamLengthPercent_memory.append(JamLengthPercent)
        Speed_memory.append(Speed)
        throughput_onnut_memory.append(throughput_onnut)
        throughput_virtual_memory.append(throughput_virtual)
        travel_time_onnut_memory.append(travel_time_onnut)
        travel_time_virtual_memory.append(travel_time_virtual)
        waiting_time_onnut_memory.append(waiting_time_onnut)
        waiting_time_virtual_memory.append(waiting_time_virtual)
        teleport_number_memory.append(teleport_number)


        #backlog = sum([traci.lanearea.getLastStepVehicleNumber(i) for i in ONNUT_backlog])
        # backlog = sum([traci.lanearea.getLastStepVehicleNumber(i) for i in list(namedetector_df['detector_id'])])
        
#         print(JamLengthMeter)
#         print(Speed)
#         print(state)
        selected_traffic_signal_phases = new_selected_traffic_signal_phases
        list_greentime_duration = new_greentime_duration 

        # with open( "./onnut_police.csv" , 'a', newline='') as csv_file:
        #     header = ['throughput_onnut','throughput_virtual','travel_time_onnut','travel_time_virtual','waiting_time_onnut','waiting_time_virtual','backlog',"meanSpeed","JamLengthPercent" ]

        #     writer = csv.DictWriter(csv_file, fieldnames = header)
        #     writer.writerow({'throughput_onnut': throughput_onnut , 'throughput_virtual' : throughput_virtual, \
        #                     'travel_time_onnut': travel_time_onnut, 'travel_time_virtual': travel_time_virtual, \
        #                     'waiting_time_onnut' : waiting_time_onnut, 'waiting_time_virtual' : waiting_time_virtual,
        #                     'backlog': backlog,"meanSpeed":Speed,"JamLengthPercent":JamLengthPercent
        #                     })

#         with open('output.csv', mode='w', newline='') as csv_file:
#             writer = csv.writer(csv_file)
#             writer.writerow(['throughput'])
#             writer.writerows([throughput_memory])
#             writer.writerow(['Jam'])
#             writer.writerows([JamLengthPercent_memory])
            
    # plt.figure(2)
    # plt.clf() 
    # plt.plot(throughput_onnut_memory)
    # plt.plot(throughput_virtual_memory)
    
#     line_up, = plt.plot(JamLengthMeter_memory)
#     line_mid,  =  plt.plot(Speed_memory)
#     line_down, = plt.plot(throughput_memory)
#     plt.legend([line_up, line_down], ['JamLengthMeter', 'Speed','throughput'])
    # plt.pause(0.001)
    traci.close()
    return action_onnut,action_virtual, \
        throughput_onnut_memory, throughput_virtual_memory,\
        travel_time_onnut_memory,travel_time_virtual_memory,\
            waiting_time_onnut_memory,waiting_time_virtual_memory,\
                teleport_number_memory

In [24]:
number_run = 100

for i in range(number_run) :

    action_onnut,action_virtual, throughput_onnut_memory, throughput_virtual_memory \
        ,travel_time_onnut_memory,travel_time_virtual_memory \
            ,waiting_time_onnut_memory,waiting_time_virtual_memory,teleport_number_memory = episode0(threshold, duration,random_seed=i) 

    baseline_df = pd.DataFrame(
        {'action_onnut' : action_onnut,
        'action_virtual' : action_virtual,
        'reward_onnut': throughput_onnut_memory,
        'reward_virtual':throughput_virtual_memory,
        'travel_time_onnut':travel_time_onnut_memory,
        'travel_time_virtual':travel_time_virtual_memory,
        'teleport_number' : teleport_number_memory
        })
    # os.chdir('C:\\Users\\Asus\\OneDrive - Chulalongkorn University\\Desktop\\Senior Project\\Experiments-with-DQN-Onnut-27042022\\Experiments-with-DQN-Onnut-21042022\\multi-agent-dqn-main\\baseline_output')
    baseline_df.to_csv('/akebaseline/tuned_police/tuned_police_output_run{}.csv'.format(i))

# print(action_onnut)
# print(action_virtual)

OSError: Cannot save file into a non-existent directory: '\akebaseline\tuned_police'

In [ ]:
baseline_df['reward_virtual'].sum()

12355

In [ ]:
# try :
#     action,throughput_onnut_memory,throughput_virtual_memory = episode0(threshold, duration) 
#     print(action)
# finally :
#     print("SUMO closing")
#     traci.close()

In [ ]:
# traci.close() #using for closing SUMO properly

In [ ]:
count

{'ONNUT_FP1_END_FLOW': 0,
 'ONNUT_FP2_END_FLOW': 136,
 'ONNUT_FP1_END_UNJAM': 141,
 'ONNUT_FP1_END_MAXGREEN': 0,
 'ONNUT_FP2_END_MAXGREEN': 5,
 'ONNUT_FP1_END_MINGREEN': 282,
 'ONNUT_FP2_END_MINGREEN': 282,
 'VIRTUAL_ONNUT_ASSOCIATE': 537,
 'VIRTUAL_FP1_END_MINGREEN': 65,
 'VIRTUAL_FP1_END_MAXGREEN': 91,
 'VIRTUAL_FP1_END_UNJAM_17': 0,
 'VIRTUAL_FP1_END_UNJAM_13_14': 26,
 'VIRTUAL_FP1_END_UNJAM_BANGNA': 38}

In [ ]:
# baseline_df = pd.DataFrame(
#     {'reward_onnut': throughput_onnut_memory,
#      'reward_virtual':throughput_virtual_memory,
#      'travel_time_onnut':travel_time_onnut_memory,
#      'travel_time_virtual':travel_time_virtual_memory
#     })
# os.chdir('C:\\Users\\Asus\\OneDrive - Chulalongkorn University\\Desktop\\Senior Project\\Experiments-with-DQN-Onnut-27042022\\Experiments-with-DQN-Onnut-21042022\\multi-agent-dqn-main')
# baseline_df.to_csv('baseline_output.csv')

In [ ]:
# # Onnut intersection phase
# print(traci.trafficlight.getCompleteRedYellowGreenDefinition('cluster_1088409501_272206263_5136790697_70702637'))

# print(traci.trafficlight.getRedYellowGreenState('cluster_1088409501_272206263_5136790697_70702637'))


C:\Users\Asus\AppData\Local\Temp\ipykernel_728\217621775.py:2: UserWarning: Call to deprecated function getCompleteRedYellowGreenDefinition, use getAllProgramLogics instead.
  print(traci.trafficlight.getCompleteRedYellowGreenDefinition('cluster_1088409501_272206263_5136790697_70702637'))


FatalTraCIError: Connection already closed.

In [ ]:
# # Virtual intersection phase
# print(traci.trafficlight.getCompleteRedYellowGreenDefinition('gneJ42'))

# print(traci.trafficlight.getRedYellowGreenState('gneJ42'))

(Logic(programID='0', type=0, currentPhaseIndex=2, phases=(Phase(duration=135.0, state='rrrgGGGGGrrGrrr', minDur=135.0, maxDur=135.0, next=()), Phase(duration=7.5, state='rrGGrrGGGrrGrrr', minDur=7.5, maxDur=7.5, next=()), Phase(duration=7.5, state='rrrgrrrGGGrGrrr', minDur=7.5, maxDur=7.5, next=())), subParameter={}),)
rrrgrrrGGGrGrrr


C:\Users\Asus\AppData\Local\Temp/ipykernel_3956/2742542155.py:2: UserWarning: Call to deprecated function getCompleteRedYellowGreenDefinition, use getAllProgramLogics instead.
  print(traci.trafficlight.getCompleteRedYellowGreenDefinition('gneJ42'))
